In [1]:
import shutil
import tqdm
import numpy as np
import cv2
import os
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
import numpy as np
import os, sys
import pickle, functools, operator
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import joblib
from tensorflow.keras.utils import to_categorical
from keras.models import Model, load_model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
import json
import random
from keras.utils.vis_utils import plot_model
import tensorflow as tf
import datetime

from pandas import read_csv
import numpy as np
from keras import Model
from keras.layers import Layer
import keras.backend as K
from keras.layers import Input, Dense, SimpleRNN , Concatenate
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.metrics import mean_squared_error

import random
random.seed(42)

In [2]:
## 02. Train & Test Split Data

with open('./training_data/AllVideoDescriptions.txt') as f:
    lines = f.readlines()

all_videos = list(set([l.split(' ')[0] for l in lines]))

with open('./train_split.txt','r') as f:
    train_l = f.readlines()
train_l = [l.replace('\n','') for l in train_l]

with open('./test_split.txt','r') as f:
    test_l = f.readlines()
test_l = [l.replace('\n','') for l in test_l]

with open('./val_split.txt','r') as f:
    val_l = f.readlines()
val_l = [l.replace('\n','') for l in val_l]

len(train_l), len(test_l) , len(val_l)

scripts = {}
for l in lines:
    id = l.split(' ')[0]
    script = ' '.join(l.split(' ')[1:])
    if id in scripts:
      scripts[id].append(script)
    else:
      scripts[id] = [script]

training_list = []
validation_list = []
test_list = [] 
vocab_list = []

for y in train_l:
  for caption in scripts[y][0:1]:
    caption = "<bos> " + caption + " <eos>"

    training_list.append([caption, y])

for y in val_l:
  for caption in scripts[y][0:1]:
    caption = "<bos> " + caption + " <eos>"

    validation_list.append([caption, y])
    
for y in test_l:
  for caption in scripts[y][0:1]:
    caption = "<bos> " + caption + " <eos>"

    test_list.append([caption, y])

train_list = training_list + validation_list + test_list

print(len(train_list))
random.shuffle(training_list)
random.shuffle(validation_list)
random.shuffle(test_list)

for train in training_list:
    vocab_list.append(train[0])
# Tokenizing the words
tokenizer = Tokenizer(num_words=1500)
tokenizer.fit_on_texts(vocab_list)
print(len(tokenizer.word_index))
x_data = {}
TRAIN_FEATURE_DIR = os.path.join('training_data', 'features_dir')
# Loading all the numpy arrays at once and saving them in a dictionary
for filename in os.listdir(TRAIN_FEATURE_DIR):
    f = np.load(os.path.join(TRAIN_FEATURE_DIR, filename))
    x_data[filename[:-4]] = f
print(len(training_list))
print(len(validation_list))
len(x_data)

1970
1702
1200
100


1970

In [31]:
print(f"Number of video:{len(all_videos)}")


print(f"Average description # per video:{np.mean([len(scripts[k]) for k in scripts])}")
print(f"All description # per video:{sum([len(scripts[k]) for k in scripts])}")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(vocab_list)
print(f"Number of Token: {len(tokenizer.index_word)}")

Number of video:1970
Average description # per video:41.028934010152284
All description # per video:80827
Number of Token: 1702
